In [1]:
from openai import OpenAI
import base64
import requests
import json
from IPython.display import display, Markdown

client = OpenAI()  # 初始化 OpenAI 客户端

def query_base64_image_description(image_path, prompt, max_tokens=1000):

    # 实现 Base64 编码
    def encode_image(path):
        with open(path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')

    # 获取图像的 Base64 编码字符串
    base64_image = encode_image(image_path)

    # 构造请求的 HTTP Header
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {client.api_key}"
    }

    # 构造请求的负载
    payload = {
        "model": "gpt-4-turbo",
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
                ]
            }
        ],
        "max_tokens": max_tokens
    }

    # 发送 HTTP 请求
    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

    # 检查响应并提取所需的 content 字段
    if response.status_code == 200:
        response_data = response.json()
        content = response_data['choices'][0]['message']['content']
        return content
    else:
        return f"Error: {response.status_code}, {response.text}"

In [2]:
def picture_explained(image_path, prompt="解释下图里的内容？"):
    content = query_base64_image_description(image_path, prompt)
    display(Markdown(content))

In [8]:
prompt = """这是一个操作系统相关的选择题，请先识别出图片中的文字，然后分析这段文字描述的问题，最后给出你的答案和作答理由。
要求输出格式包含‘文字内容’和‘问题解答’两个模块，‘文字内容’请尽可能保持其在图片中的原有的格式，‘问题解答’需要包含答案和作答理由两部分。"""

picture_explained("../../openai_api/images/gpt-4v.jpg", prompt)

文字内容：
下列关于虚拟内存不换页技术的叙述中，哪项错误？
A. 虚拟内存是为了使得互操作系统可以使用更多的内存地址
B. 避免虚拟内存换页可以减少虚拟内存的使用和提高系统的响应速度。
C. 一个虚拟内存的换页需求变化较难预测与控制
D. 脏页更换算法(MMU)在虚拟内存更换过程中起主导作用。

问题解答：
答案：A

作答理由：
选项A表述“虚拟内存是为了使得操作系统可以使用更多的内存地址”，实际上，虚拟内存的主要目的是让操作系统能高效管理有限的物理内存资源，并不是直接为了增加内存地址的数量，而是通过地址转换机制（物理地址与虚拟地址之间的映射），让操作系统和程序认为有更多的内存可用。

选项B提到不换页可以减少虚拟内存使用和提高系统响应速度，这部分是正确的，因为频繁的页换入换出会导致较高的系统开销和延迟。

选项C指出虚拟内存的换页需求变化难以预测和控制也是正确的，因为页访问的模式往往依赖于程序的具体行为，其动态性使得预测变得复杂。

选项D中的“脏页更换算法(MMU)”描述不准确。MMU（内存管理单元）主要负责虚拟地址到物理地址的转换，而具体页的替换算法通常由操作系统中的页面替换算法部分来处理，而不是MMU。不过，因为MMU涉及到脏页（修改过需写回的页）的标识及处理，可能造成混淆，但核心替换算法逻辑不由MMU执行。选项D有一定的表述不清，但相比A选项的错误，D选项表述的错误程度较轻。